In [1]:
import pandas as pd
import numpy as np
from unidecode import unidecode

from ydata_profiling import ProfileReport
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import OneHotEncoder

C:\Users\thami\Documents\GitHub\TCC agrupamento\agrupamento-tcc\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Funções

In [2]:
def remove_acentos(texto):
    return unidecode(str(texto))

In [3]:
def minusculo(texto):
    texto = texto.replace(' ', '_')
    return str(texto).lower()

#### Leitura

In [4]:
df_2021 = pd.read_csv('dados/microdados_2022/dados/MICRODADOS_CADASTRO_CURSOS_2022.CSV',encoding='ISO-8859-1',sep=';')

C:\Users\thami\AppData\Local\Temp\ipykernel_18804\4259865736.py:1: DtypeWarning: Columns (1,3,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2021 = pd.read_csv('dados/microdados_2022/dados/MICRODADOS_CADASTRO_CURSOS_2022.CSV',encoding='ISO-8859-1',sep=';')


In [5]:
df_indicadores = pd.read_csv('dados_processados/indicadores.csv', sep=',')

In [6]:
#profile = ProfileReport(df_2021, title='Relatório de Dados Mínimo', minimal=True)

In [7]:
#profile.to_file('microdados.html')

#### Limpeza e ajuste de dados

In [8]:
df_2021.shape

(573019, 200)

In [9]:
df_2021 = df_2021.dropna()

In [10]:
df_2021.shape

(563416, 200)

In [11]:
df_2021['TP_MODALIDADE_ENSINO'].unique()

array([1, 2], dtype=int64)

In [12]:
df_2021 = df_2021.query('TP_MODALIDADE_ENSINO == 1').reset_index(drop=True).copy()

In [13]:
df_2021['TP_MODALIDADE_ENSINO'].unique()

array([1], dtype=int64)

In [14]:
cursos_ufrpe = df_2021[df_2021['CO_IES'] == 587][['CO_CURSO', 'NO_CURSO']]

In [15]:
cursos_ufrpe.head()

,CO_CURSO,NO_CURSO
8629,1516690,Engenharia De Computação
8630,1516692,Engenharia De Controle E Automação
8631,1516689,Engenharia Hídrica
8632,1516693,Engenharia Química
8664,1270379,Engenharia Civil


In [16]:
cursos_ufrpe.to_csv('dados_processados/cursos_ufrpe.csv', sep=';', index=False)

In [17]:
df_2021

,NU_ANO_CENSO,NO_REGIAO,CO_REGIAO,NO_UF,SG_UF,CO_UF,NO_MUNICIPIO,CO_MUNICIPIO,IN_CAPITAL,TP_DIMENSAO,...,QT_MAT_APOIO_SOCIAL,QT_CONC_APOIO_SOCIAL,QT_ATIV_EXTRACURRICULAR,QT_ING_ATIV_EXTRACURRICULAR,QT_MAT_ATIV_EXTRACURRICULAR,QT_CONC_ATIV_EXTRACURRICULAR,QT_MOB_ACADEMICA,QT_ING_MOB_ACADEMICA,QT_MAT_MOB_ACADEMICA,QT_CONC_MOB_ACADEMICA
0,2022,Centro-Oeste,5,Distrito Federal,DF,53,Brasília,5300108,1,1,...,4,0,1,1,1,0,0,0,0,0
1,2022,Centro-Oeste,5,Distrito Federal,DF,53,Brasília,5300108,1,1,...,0,0,0,0,0,0,0,0,0,0
2,2022,Centro-Oeste,5,Distrito Federal,DF,53,Brasília,5300108,1,1,...,0,0,0,0,0,0,0,0,0,0
3,2022,Centro-Oeste,5,Distrito Federal,DF,53,Brasília,5300108,1,1,...,71,0,168,65,66,0,0,0,0,0
4,2022,Centro-Oeste,5,Distrito Federal,DF,53,Brasília,5300108,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36084,2022,Sul,4,Santa Catarina,SC,42,Xaxim,4219705,0,1,...,0,0,46,18,7,7,0,0,0,0
36085,2022,Sul,4,Santa Catarina,SC,42,Xaxim,4219705,0,1,...,0,0,30,3,0,0,0,0,0,0
36086,2022,Sul,4,Santa Catarina,SC,42,Xaxim,4219705,0,1,...,0,0,75,14,18,18,0,0,0,0
36087,2022,Sul,4,Santa Catarina,SC,42,Xaxim,4219705,0,1,...,0,0,10,0,7,7,0,0,0,0


#### Ajustes dataframe microdados

In [18]:
df_2021.replace({'TP_GRAU_ACADEMICO':{1:'bacharelado',2:'licenciatura',3:'tecnologico',4:'bacharelado e licenciatura'}},inplace=True)

In [19]:
df_2021.columns = df_2021.columns.str.lower()

In [20]:
colunas_categoricas = df_2021.select_dtypes(include=['object']).columns
for coluna in colunas_categoricas:
    df_2021[coluna] = df_2021[coluna].apply(remove_acentos)
    df_2021[coluna] = df_2021[coluna].apply(minusculo)

In [21]:
df_2021

,nu_ano_censo,no_regiao,co_regiao,no_uf,sg_uf,co_uf,no_municipio,co_municipio,in_capital,tp_dimensao,...,qt_mat_apoio_social,qt_conc_apoio_social,qt_ativ_extracurricular,qt_ing_ativ_extracurricular,qt_mat_ativ_extracurricular,qt_conc_ativ_extracurricular,qt_mob_academica,qt_ing_mob_academica,qt_mat_mob_academica,qt_conc_mob_academica
0,2022,centro-oeste,5,distrito_federal,df,53,brasilia,5300108,1,1,...,4,0,1,1,1,0,0,0,0,0
1,2022,centro-oeste,5,distrito_federal,df,53,brasilia,5300108,1,1,...,0,0,0,0,0,0,0,0,0,0
2,2022,centro-oeste,5,distrito_federal,df,53,brasilia,5300108,1,1,...,0,0,0,0,0,0,0,0,0,0
3,2022,centro-oeste,5,distrito_federal,df,53,brasilia,5300108,1,1,...,71,0,168,65,66,0,0,0,0,0
4,2022,centro-oeste,5,distrito_federal,df,53,brasilia,5300108,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36084,2022,sul,4,santa_catarina,sc,42,xaxim,4219705,0,1,...,0,0,46,18,7,7,0,0,0,0
36085,2022,sul,4,santa_catarina,sc,42,xaxim,4219705,0,1,...,0,0,30,3,0,0,0,0,0,0
36086,2022,sul,4,santa_catarina,sc,42,xaxim,4219705,0,1,...,0,0,75,14,18,18,0,0,0,0
36087,2022,sul,4,santa_catarina,sc,42,xaxim,4219705,0,1,...,0,0,10,0,7,7,0,0,0,0


In [22]:
print(df_indicadores.dtypes)

tp_grau_academico     object
co_curso               int64
enade_faixa          float64
cpc_faixa            float64
dtype: object


In [23]:
df_indicadores.columns

Index(['tp_grau_academico', 'co_curso', 'enade_faixa', 'cpc_faixa'], dtype='object')

In [24]:
#Para evitar enade_faixa e cpc_faixa NaN não foi utilizado o left no merge
resultado = pd.merge(df_2021, df_indicadores[['co_curso', 'enade_faixa', 'cpc_faixa']], on='co_curso')

In [25]:
resultado

,nu_ano_censo,no_regiao,co_regiao,no_uf,sg_uf,co_uf,no_municipio,co_municipio,in_capital,tp_dimensao,...,qt_ativ_extracurricular,qt_ing_ativ_extracurricular,qt_mat_ativ_extracurricular,qt_conc_ativ_extracurricular,qt_mob_academica,qt_ing_mob_academica,qt_mat_mob_academica,qt_conc_mob_academica,enade_faixa,cpc_faixa
0,2022,centro-oeste,5,distrito_federal,df,53,brasilia,5300108,1,1,...,550,37,543,52,4,0,4,4,5.0,5.0
1,2022,centro-oeste,5,distrito_federal,df,53,brasilia,5300108,1,1,...,294,6,285,27,0,0,0,0,4.0,4.0
2,2022,centro-oeste,5,distrito_federal,df,53,brasilia,5300108,1,1,...,219,19,214,12,0,0,0,0,4.0,4.0
3,2022,centro-oeste,5,distrito_federal,df,53,brasilia,5300108,1,1,...,207,7,202,10,0,0,0,0,5.0,5.0
4,2022,centro-oeste,5,distrito_federal,df,53,brasilia,5300108,1,1,...,121,6,118,4,2,0,2,0,5.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1259,2022,sul,4,santa_catarina,sc,42,ibirama,4206900,0,1,...,89,16,82,18,1,0,1,0,5.0,4.0
1260,2022,sul,4,santa_catarina,sc,42,ibirama,4206900,0,1,...,3,0,3,0,0,0,0,0,2.0,4.0
1261,2022,sul,4,santa_catarina,sc,42,jaragua_do_sul,4208906,0,1,...,3,0,3,0,0,0,0,0,4.0,4.0
1262,2022,sul,4,santa_catarina,sc,42,palhoca,4211900,0,1,...,0,0,0,0,0,0,0,0,3.0,3.0


In [26]:
resultado.shape

(1264, 202)

In [27]:
resultado.describe()

,nu_ano_censo,co_regiao,co_uf,co_municipio,in_capital,tp_dimensao,tp_organizacao_academica,tp_categoria_administrativa,tp_rede,co_ies,...,qt_ativ_extracurricular,qt_ing_ativ_extracurricular,qt_mat_ativ_extracurricular,qt_conc_ativ_extracurricular,qt_mob_academica,qt_ing_mob_academica,qt_mat_mob_academica,qt_conc_mob_academica,enade_faixa,cpc_faixa
count,1264.0,1264.000000,1264.000000,1.264000e+03,1264.000000,1264.0,1264.000000,1264.000000,1264.0,1264.000000,...,1264.000000,1264.000000,1264.000000,1264.000000,1264.000000,1264.000000,1264.000000,1264.000000,1264.000000,1264.000000
mean,2022.0,3.003165,33.352848,3.350392e+06,0.261076,1.0,1.636867,1.679589,1.0,2888.533228,...,46.250791,4.627373,43.735759,6.397943,0.381329,0.005538,0.376582,0.033228,3.608386,3.478639
std,0.0,1.160851,10.536842,1.054312e+06,0.439395,0.0,1.121418,1.188235,0.0,5476.437439,...,95.992459,14.626614,90.875872,17.262329,1.742805,0.084233,1.721447,0.246278,1.019661,0.997790
min,2022.0,1.000000,11.000000,1.100049e+06,0.000000,1.0,1.000000,1.000000,1.0,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2022.0,2.000000,26.000000,2.606653e+06,0.000000,1.0,1.000000,1.000000,1.0,568.750000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,3.000000
50%,2022.0,3.000000,33.000000,3.303302e+06,0.000000,1.0,1.000000,1.000000,1.0,609.000000,...,14.000000,0.000000,13.000000,1.000000,0.000000,0.000000,0.000000,0.000000,4.000000,4.000000
75%,2022.0,4.000000,41.000000,4.117602e+06,1.000000,1.0,2.000000,2.000000,1.0,1820.000000,...,49.000000,3.000000,48.000000,7.000000,0.000000,0.000000,0.000000,0.000000,4.000000,4.000000
max,2022.0,5.000000,53.000000,5.300108e+06,1.000000,1.0,5.000000,7.000000,1.0,25352.000000,...,1215.000000,339.000000,1177.000000,418.000000,37.000000,2.000000,37.000000,4.000000,5.000000,5.000000


In [28]:
resultado['enade_faixa'].unique()

array([5., 4., 3., 2., 0., 1.])

In [29]:
print(resultado.dtypes)

nu_ano_censo               int64
no_regiao                 object
co_regiao                  int64
no_uf                     object
sg_uf                     object
                          ...   
qt_ing_mob_academica       int64
qt_mat_mob_academica       int64
qt_conc_mob_academica      int64
enade_faixa              float64
cpc_faixa                float64
Length: 202, dtype: object


In [30]:
if resultado.isnull().all().any():
    print('Há pelo menos uma coluna com dados ausentes.')
else:
    print('Não há colunas com dados ausentes.')

Não há colunas com dados ausentes.


In [31]:
df = resultado.drop(['no_regiao', 'no_uf', 'sg_uf', 'no_cine_rotulo', 'co_cine_rotulo', 'no_cine_area_especifica', 'no_cine_area_detalhada',
                     'co_cine_area_geral', 'co_cine_area_especifica', 'co_cine_area_detalhada', 'co_municipio'],axis=1)

In [32]:
df.shape

(1264, 191)

#### Tratamento

In [33]:
#Primeiro passo é identificar os outliers e substituí-los

In [34]:
def detect_outliers_iqr(data, threshold=1.5):
    """
    Detecta outliers em uma série de dados usando o método do intervalo interquartil (IQR).
    
    Parâmetros:
        data (pd.Series): A série de dados para verificar outliers.
        threshold (float): O fator multiplicativo do IQR usado para definir o limite para outliers.
        
    Retorna:
        outliers (int): O número de amostras com outliers.
    """
    q1 = data.quantile(0.25)
    q3 = data.quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - threshold * iqr
    upper_bound = q3 + threshold * iqr
    
    outliers = data[(data < lower_bound) | (data > upper_bound)]
    return len(outliers)

In [35]:
# Calcula a quantidade de amostras com outliers para cada coluna numérica
outliers_count = {}
for column in df.select_dtypes(include=['number']):
    outliers_count[column] = detect_outliers_iqr(df[column])

# Exibe a quantidade de amostras com outliers por coluna
for column, count in outliers_count.items():
    print(f"Coluna '{column}': {count} amostras com outliers")

Coluna 'nu_ano_censo': 0 amostras com outliers
Coluna 'co_regiao': 0 amostras com outliers
Coluna 'co_uf': 0 amostras com outliers
Coluna 'in_capital': 0 amostras com outliers
Coluna 'tp_dimensao': 0 amostras com outliers
Coluna 'tp_organizacao_academica': 167 amostras com outliers
Coluna 'tp_categoria_administrativa': 45 amostras com outliers
Coluna 'tp_rede': 0 amostras com outliers
Coluna 'co_ies': 234 amostras com outliers
Coluna 'co_curso': 26 amostras com outliers
Coluna 'in_gratuito': 109 amostras com outliers
Coluna 'tp_modalidade_ensino': 0 amostras com outliers
Coluna 'tp_nivel_academico': 0 amostras com outliers
Coluna 'qt_curso': 0 amostras com outliers
Coluna 'qt_vg_total': 79 amostras com outliers
Coluna 'qt_vg_total_diurno': 29 amostras com outliers
Coluna 'qt_vg_total_noturno': 28 amostras com outliers
Coluna 'qt_vg_total_ead': 0 amostras com outliers
Coluna 'qt_vg_nova': 63 amostras com outliers
Coluna 'qt_vg_proc_seletivo': 0 amostras com outliers
Coluna 'qt_vg_remane

In [36]:
def replace_outliers_iqr(data, threshold=1.5):
    """
    Substitui outliers em uma série de dados pelos limites inferior ou superior do IQR.
    
    Parâmetros:
        data (pd.Series): A série de dados para substituir outliers.
        threshold (float): O fator multiplicativo do IQR usado para definir o limite para outliers.
        
    Retorna:
        data_with_replaced_outliers (pd.Series): A série de dados com outliers substituídos.
    """
    q1 = data.quantile(0.25)
    q3 = data.quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - threshold * iqr
    upper_bound = q3 + threshold * iqr
    
    data_with_replaced_outliers = data.copy()
    data_with_replaced_outliers[data_with_replaced_outliers < lower_bound] = lower_bound
    data_with_replaced_outliers[data_with_replaced_outliers > upper_bound] = upper_bound
    
    return data_with_replaced_outliers

In [37]:
# Substitui os outliers por limites do IQR para cada coluna numérica
for column in df.select_dtypes(include=['number']):
    df[column] = replace_outliers_iqr(df[column])

C:\Users\thami\AppData\Local\Temp\ipykernel_18804\950015012.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data_with_replaced_outliers[data_with_replaced_outliers > upper_bound] = upper_bound
C:\Users\thami\AppData\Local\Temp\ipykernel_18804\950015012.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data_with_replaced_outliers[data_with_replaced_outliers > upper_bound] = upper_bound
C:\Users\thami\AppData\Local\Temp\ipykernel_18804\950015012.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3696.875' has dtype incompatible with int64, please ex

In [38]:
# Calcula a quantidade de amostras com outliers para cada coluna numérica
outliers_count = {}
for column in df.select_dtypes(include=['number']):
    outliers_count[column] = detect_outliers_iqr(df[column])

# Exibe a quantidade de amostras com outliers por coluna
for column, count in outliers_count.items():
    print(f"Coluna '{column}': {count} amostras com outliers")

Coluna 'nu_ano_censo': 0 amostras com outliers
Coluna 'co_regiao': 0 amostras com outliers
Coluna 'co_uf': 0 amostras com outliers
Coluna 'in_capital': 0 amostras com outliers
Coluna 'tp_dimensao': 0 amostras com outliers
Coluna 'tp_organizacao_academica': 0 amostras com outliers
Coluna 'tp_categoria_administrativa': 0 amostras com outliers
Coluna 'tp_rede': 0 amostras com outliers
Coluna 'co_ies': 0 amostras com outliers
Coluna 'co_curso': 0 amostras com outliers
Coluna 'in_gratuito': 0 amostras com outliers
Coluna 'tp_modalidade_ensino': 0 amostras com outliers
Coluna 'tp_nivel_academico': 0 amostras com outliers
Coluna 'qt_curso': 0 amostras com outliers
Coluna 'qt_vg_total': 0 amostras com outliers
Coluna 'qt_vg_total_diurno': 0 amostras com outliers
Coluna 'qt_vg_total_noturno': 0 amostras com outliers
Coluna 'qt_vg_total_ead': 0 amostras com outliers
Coluna 'qt_vg_nova': 0 amostras com outliers
Coluna 'qt_vg_proc_seletivo': 0 amostras com outliers
Coluna 'qt_vg_remanesc': 0 amost

In [39]:
df

,nu_ano_censo,co_regiao,co_uf,no_municipio,in_capital,tp_dimensao,tp_organizacao_academica,tp_categoria_administrativa,tp_rede,co_ies,...,qt_ativ_extracurricular,qt_ing_ativ_extracurricular,qt_mat_ativ_extracurricular,qt_conc_ativ_extracurricular,qt_mob_academica,qt_ing_mob_academica,qt_mat_mob_academica,qt_conc_mob_academica,enade_faixa,cpc_faixa
0,2022,5,53,brasilia,1,1,1.0,1.0,1,2.000,...,122.5,7.5,120,17.5,0,0,0,0,5.0,5.0
1,2022,5,53,brasilia,1,1,1.0,1.0,1,2.000,...,122.5,6.0,120,17.5,0,0,0,0,4.0,4.0
2,2022,5,53,brasilia,1,1,1.0,1.0,1,2.000,...,122.5,7.5,120,12.0,0,0,0,0,4.0,4.0
3,2022,5,53,brasilia,1,1,1.0,1.0,1,2.000,...,122.5,7.0,120,10.0,0,0,0,0,5.0,5.0
4,2022,5,53,brasilia,1,1,1.0,1.0,1,2.000,...,121.0,6.0,118,4.0,0,0,0,0,5.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1259,2022,4,42,ibirama,0,1,1.0,2.0,1,43.000,...,89.0,7.5,82,17.5,0,0,0,0,5.0,4.0
1260,2022,4,42,ibirama,0,1,3.5,1.0,1,3696.875,...,3.0,0.0,3,0.0,0,0,0,0,2.0,4.0
1261,2022,4,42,jaragua_do_sul,0,1,3.5,1.0,1,3162.000,...,3.0,0.0,3,0.0,0,0,0,0,4.0,4.0
1262,2022,4,42,palhoca,0,1,3.0,3.0,1,3696.875,...,0.0,0.0,0,0.0,0,0,0,0,3.0,3.0


In [40]:
df.shape

(1264, 191)

In [41]:
df1 = df.drop(['no_curso','no_municipio','no_cine_area_geral','tp_grau_academico','enade_faixa', 'cpc_faixa'],axis=1)

In [42]:
# Remover todas as colunas com variância zero

In [43]:
sel = VarianceThreshold(threshold=(0))
df_selected = sel.fit_transform(df1)
cols_selected = df1.columns[sel.get_support(indices=True)]
df_selected = pd.DataFrame(df_selected, columns=cols_selected)

In [44]:
df_selected

,co_regiao,co_uf,in_capital,tp_organizacao_academica,tp_categoria_administrativa,co_ies,co_curso,qt_vg_total,qt_vg_total_diurno,qt_vg_total_noturno,...,qt_conc_procescpublica,qt_conc_procescprivada,qt_apoio_social,qt_ing_apoio_social,qt_mat_apoio_social,qt_conc_apoio_social,qt_ativ_extracurricular,qt_ing_ativ_extracurricular,qt_mat_ativ_extracurricular,qt_conc_ativ_extracurricular
0,5.0,53.0,1.0,1.0,1.0,2.000,145.0,205.0,139.0,132.0,...,55.0,37.0,107.5,15.0,100.0,9.0,122.5,7.5,120.0,17.5
1,5.0,53.0,1.0,1.0,1.0,2.000,149.0,159.0,159.0,0.0,...,55.0,37.0,107.5,15.0,100.0,10.0,122.5,6.0,120.0,17.5
2,5.0,53.0,1.0,1.0,1.0,2.000,167.0,144.0,0.0,144.0,...,43.0,37.0,104.0,15.0,100.0,6.0,122.5,7.5,120.0,12.0
3,5.0,53.0,1.0,1.0,1.0,2.000,122.0,104.0,104.0,0.0,...,21.0,37.0,105.0,15.0,96.0,6.0,122.5,7.0,120.0,10.0
4,5.0,53.0,1.0,1.0,1.0,2.000,31381.0,71.0,71.0,0.0,...,14.0,10.0,75.0,12.0,73.0,2.0,121.0,6.0,118.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1259,4.0,42.0,0.0,1.0,2.0,43.000,53596.0,100.0,0.0,100.0,...,27.0,5.0,5.0,1.0,5.0,0.0,89.0,7.5,82.0,17.5
1260,4.0,42.0,0.0,3.5,1.0,3696.875,1283114.0,71.0,0.0,71.0,...,6.0,1.0,9.0,0.0,9.0,1.0,3.0,0.0,3.0,0.0
1261,4.0,42.0,0.0,3.5,1.0,3162.000,1483743.0,120.0,0.0,120.0,...,4.0,1.0,43.0,11.0,29.0,2.0,3.0,0.0,3.0,0.0
1262,4.0,42.0,0.0,3.0,3.0,3696.875,122406.0,179.0,54.0,125.0,...,55.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
df_selected.shape

(1264, 107)

In [46]:
# Padronizando os dados com StandardScaler

In [47]:
scaler = StandardScaler()
standard_data = scaler.fit_transform(df_selected.drop('co_curso', axis=1))

In [48]:
df_processed = pd.DataFrame(standard_data, columns=df_selected.drop('co_curso', axis=1).columns)

In [49]:
df_processed

,co_regiao,co_uf,in_capital,tp_organizacao_academica,tp_categoria_administrativa,co_ies,qt_vg_total,qt_vg_total_diurno,qt_vg_total_noturno,qt_vg_nova,...,qt_conc_procescpublica,qt_conc_procescprivada,qt_apoio_social,qt_ing_apoio_social,qt_mat_apoio_social,qt_conc_apoio_social,qt_ativ_extracurricular,qt_ing_ativ_extracurricular,qt_mat_ativ_extracurricular,qt_conc_ativ_extracurricular
0,1.720829,1.865353,1.682350,-0.581020,-0.780599,-0.964041,2.278923,2.207315,1.653686,2.287025,...,2.214107,2.276417,2.260930,2.143325,2.272136,1.880136,2.181427,2.044026,2.210957,2.213229
1,1.720829,1.865353,1.682350,-0.581020,-0.780599,-0.964041,1.384074,2.643753,-1.125522,1.678840,...,2.214107,2.276417,2.260930,2.143325,2.272136,2.166944,2.181427,1.498223,2.210957,2.213229
2,1.720829,1.865353,1.682350,-0.581020,-0.780599,-0.964041,1.092275,-0.825925,1.906342,1.192293,...,1.429132,2.276417,2.161761,2.143325,2.272136,1.019712,2.181427,2.044026,2.210957,1.287093
3,1.720829,1.865353,1.682350,-0.581020,-0.780599,-0.964041,0.314145,1.443550,-1.125522,0.705745,...,-0.009988,2.276417,2.190095,2.143325,2.149907,1.019712,2.181427,1.862092,2.210957,0.950316
4,1.720829,1.865353,1.682350,-0.581020,-0.780599,-0.964041,-0.327812,0.723428,-1.125522,-0.656589,...,-0.467890,-0.057935,1.340078,1.571691,1.447091,-0.127521,2.145034,1.498223,2.161003,-0.060015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1259,0.859051,0.820984,-0.594406,-0.581020,0.625926,-0.933429,0.236332,-0.825925,0.979939,0.219197,...,0.382499,-0.490222,-0.643297,-0.524299,-0.630798,-0.701137,1.368651,2.044026,1.261840,2.213229
1260,0.859051,0.820984,-0.594406,1.977896,-0.780599,1.794645,-0.327812,-0.825925,0.369355,-0.997173,...,-0.991207,-0.836052,-0.529961,-0.714844,-0.508570,-0.414329,-0.717878,-0.684988,-0.711324,-0.733569
1261,0.859051,0.820984,-0.594406,1.977896,-0.780599,1.395294,0.625397,-0.825925,1.401031,0.219197,...,-1.122036,-0.836052,0.433392,1.381147,0.102575,-0.127521,-0.717878,-0.684988,-0.711324,-0.733569
1262,0.859051,0.820984,-0.594406,1.466113,2.032450,1.794645,1.773139,0.352456,1.506304,2.287025,...,2.214107,-0.403765,-0.784966,-0.714844,-0.783584,-0.701137,-0.790664,-0.684988,-0.786254,-0.733569


In [50]:
df_processed.shape

(1264, 106)

In [51]:
df_processed['co_curso'] = df['co_curso']
df_processed['no_curso'] = df['no_municipio'] + '_' + df['tp_grau_academico'] + '_' + df['no_curso']
df_processed['no_cine_area_geral'] = df['no_cine_area_geral']
df_processed['tp_grau_academico'] = df['tp_grau_academico']
df_processed['enade_faixa'] = df['enade_faixa']
df_processed['cpc_faixa'] = df['cpc_faixa']

In [52]:
# Calculando a correlação entre colunas e excluindo as que tem correlação maior que o limite determinado (0.8)

In [53]:
limit = 0.8
corr = df_processed.corr(numeric_only=True)
mask = np.triu(
    np.ones(corr.shape), k=1
).astype(bool)
corr_no_dial = corr.where(mask)
coll = [
    c
    for c in corr_no_dial.columns
    if any(abs(corr_no_dial[c])>limit)
]

In [54]:
coll

['co_uf',
 'qt_vg_nova',
 'qt_insc_vg_nova',
 'qt_ing',
 'qt_ing_fem',
 'qt_ing_masc',
 'qt_ing_diurno',
 'qt_ing_vg_nova',
 'qt_ing_18_24',
 'qt_ing_branca',
 'qt_mat',
 'qt_mat_fem',
 'qt_mat_masc',
 'qt_mat_diurno',
 'qt_mat_noturno',
 'qt_mat_18_24',
 'qt_mat_25_29',
 'qt_mat_30_34',
 'qt_mat_35_39',
 'qt_mat_40_49',
 'qt_mat_branca',
 'qt_mat_preta',
 'qt_mat_parda',
 'qt_conc_fem',
 'qt_conc_masc',
 'qt_conc_diurno',
 'qt_conc_18_24',
 'qt_conc_25_29',
 'qt_conc_branca',
 'qt_conc_parda',
 'qt_ing_nacbras',
 'qt_mat_nacbras',
 'qt_conc_nacbras',
 'qt_mat_deficiente',
 'qt_ing_rvredepublica',
 'qt_ing_rvetnico',
 'qt_ing_rvsocial_rf',
 'qt_mat_reserva_vaga',
 'qt_mat_rvredepublica',
 'qt_mat_rvetnico',
 'qt_mat_rvsocial_rf',
 'qt_conc_reserva_vaga',
 'qt_conc_rvredepublica',
 'qt_conc_rvetnico',
 'qt_conc_rvsocial_rf',
 'qt_ing_procescpublica',
 'qt_ing_procescprivada',
 'qt_mat_procescpublica',
 'qt_mat_procescprivada',
 'qt_conc_procescpublica',
 'qt_conc_procescprivada',
 'qt_i

In [55]:
df_processed = df_processed.drop(coll, axis=1)

In [56]:
df_processed['co_curso'] = df['co_curso']

In [57]:
df_processed.shape

(1264, 57)

In [58]:
df_processed

,co_regiao,in_capital,tp_organizacao_academica,tp_categoria_administrativa,co_ies,qt_vg_total,qt_vg_total_diurno,qt_vg_total_noturno,qt_vg_remanesc,qt_inscrito_total,...,qt_apoio_social,qt_conc_apoio_social,qt_ativ_extracurricular,qt_ing_ativ_extracurricular,co_curso,no_curso,no_cine_area_geral,tp_grau_academico,enade_faixa,cpc_faixa
0,1.720829,1.682350,-0.581020,-0.780599,-0.964041,2.278923,2.207315,1.653686,0.420073,1.187190,...,2.260930,1.880136,2.181427,2.044026,145,brasilia_bacharelado_administracao,"negocios,_administracao_e_direito",bacharelado,5.0,5.0
1,1.720829,1.682350,-0.581020,-0.780599,-0.964041,1.384074,2.643753,-1.125522,0.302673,-0.419225,...,2.260930,2.166944,2.181427,1.498223,149,brasilia_bacharelado_ciencias_contabeis,"negocios,_administracao_e_direito",bacharelado,4.0,4.0
2,1.720829,1.682350,-0.581020,-0.780599,-0.964041,1.092275,-0.825925,1.906342,0.537474,-0.357598,...,2.161761,1.019712,2.181427,2.044026,167,brasilia_bacharelado_ciencias_contabeis,"negocios,_administracao_e_direito",bacharelado,4.0,4.0
3,1.720829,1.682350,-0.581020,-0.780599,-0.964041,0.314145,1.443550,-1.125522,-0.577830,0.131311,...,2.190095,1.019712,2.181427,1.862092,122,brasilia_bacharelado_ciencias_economicas,"ciencias_sociais,_comunicacao_e_informacao",bacharelado,5.0,5.0
4,1.720829,1.682350,-0.581020,-0.780599,-0.964041,-0.327812,0.723428,-1.125522,0.772275,0.182667,...,1.340078,-0.127521,2.145034,1.498223,31381,brasilia_bacharelado_comunicacao_social_-_publ...,"negocios,_administracao_e_direito",bacharelado,5.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1259,0.859051,-0.594406,-0.581020,0.625926,-0.933429,0.236332,-0.825925,0.979939,0.361373,-0.367869,...,-0.643297,-0.701137,1.368651,2.044026,53596,ibirama_bacharelado_ciencias_contabeis,"negocios,_administracao_e_direito",bacharelado,5.0,4.0
1260,0.859051,-0.594406,1.977896,-0.780599,1.794645,-0.327812,-0.825925,0.369355,1.594078,-0.641083,...,-0.529961,-0.414329,-0.717878,-0.684988,1283114,ibirama_tecnologico_design_de_moda,artes_e_humanidades,tecnologico,2.0,4.0
1261,0.859051,-0.594406,1.977896,-0.780599,1.395294,0.625397,-0.825925,1.401031,1.535378,1.885632,...,0.433392,-0.127521,-0.717878,-0.684988,1483743,jaragua_do_sul_tecnologico_design_de_moda,artes_e_humanidades,tecnologico,4.0,4.0
1262,0.859051,-0.594406,1.466113,2.032450,1.794645,1.773139,0.352456,1.506304,-0.108229,-0.606161,...,-0.784966,-0.701137,-0.790664,-0.684988,122406,palhoca_bacharelado_administracao,"negocios,_administracao_e_direito",bacharelado,3.0,3.0


In [59]:
# Codificando as colunas categóricas usando o OneHotEncoder

In [60]:
enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

In [61]:
categoricas = ['no_cine_area_geral','tp_grau_academico', 'enade_faixa', 'cpc_faixa']

In [62]:
matriz_enc = enc.fit_transform(df_processed[categoricas])

In [63]:
df_enc = pd.DataFrame(matriz_enc, columns=enc.get_feature_names_out())

In [64]:
X = pd.concat([df_enc.reset_index(drop=True),df_processed.drop(columns=categoricas).reset_index(drop=True)],axis=1)

In [65]:
X

,no_cine_area_geral_artes_e_humanidades,"no_cine_area_geral_ciencias_sociais,_comunicacao_e_informacao","no_cine_area_geral_negocios,_administracao_e_direito",no_cine_area_geral_saude_e_bem-estar,no_cine_area_geral_servicos,tp_grau_academico_bacharelado,tp_grau_academico_tecnologico,enade_faixa_1.5,enade_faixa_2.0,enade_faixa_3.0,...,qt_ing_reserva_vaga,qt_mat_rvpdef,qt_sit_trancada,qt_sit_desvinculado,qt_apoio_social,qt_conc_apoio_social,qt_ativ_extracurricular,qt_ing_ativ_extracurricular,co_curso,no_curso
0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,2.627628,0.454924,-0.777512,2.408320,2.260930,1.880136,2.181427,2.044026,145,brasilia_bacharelado_administracao
1,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.478779,-0.115536,-0.741858,1.402383,2.260930,2.166944,2.181427,1.498223,149,brasilia_bacharelado_ciencias_contabeis
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,2.080557,-0.685997,-0.813166,2.133974,2.161761,1.019712,2.181427,2.044026,167,brasilia_bacharelado_ciencias_contabeis
3,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.807022,-0.115536,-0.813166,1.310935,2.190095,1.019712,2.181427,1.862092,122,brasilia_bacharelado_ciencias_economicas
4,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.603466,-0.685997,-0.848821,-0.335143,1.340078,-0.127521,2.145034,1.498223,31381,brasilia_bacharelado_comunicacao_social_-_publ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1259,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.877001,-0.685997,-0.385315,1.402383,-0.643297,-0.701137,1.368651,2.044026,53596,ibirama_bacharelado_ciencias_contabeis
1260,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,-0.928332,-0.685997,1.076512,-1.036251,-0.529961,-0.414329,-0.717878,-0.684988,1283114,ibirama_tecnologico_design_de_moda
1261,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.162433,-0.685997,0.541697,-0.030314,0.433392,-0.127521,-0.717878,-0.684988,1483743,jaragua_do_sul_tecnologico_design_de_moda
1262,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,2.627628,-0.685997,0.292117,2.408320,-0.784966,-0.701137,-0.790664,-0.684988,122406,palhoca_bacharelado_administracao


In [66]:
X.shape

(1264, 70)

In [67]:
X = X.drop('in_capital', axis=1)

In [68]:
X.shape

(1264, 69)

In [69]:
X.columns

Index(['no_cine_area_geral_artes_e_humanidades',
       'no_cine_area_geral_ciencias_sociais,_comunicacao_e_informacao',
       'no_cine_area_geral_negocios,_administracao_e_direito',
       'no_cine_area_geral_saude_e_bem-estar', 'no_cine_area_geral_servicos',
       'tp_grau_academico_bacharelado', 'tp_grau_academico_tecnologico',
       'enade_faixa_1.5', 'enade_faixa_2.0', 'enade_faixa_3.0',
       'enade_faixa_4.0', 'enade_faixa_5.0', 'cpc_faixa_1.5', 'cpc_faixa_2.0',
       'cpc_faixa_3.0', 'cpc_faixa_4.0', 'cpc_faixa_5.0', 'co_regiao',
       'tp_organizacao_academica', 'tp_categoria_administrativa', 'co_ies',
       'qt_vg_total', 'qt_vg_total_diurno', 'qt_vg_total_noturno',
       'qt_vg_remanesc', 'qt_inscrito_total', 'qt_inscrito_total_diurno',
       'qt_inscrito_total_noturno', 'qt_insc_vg_remanesc', 'qt_ing_noturno',
       'qt_ing_vestibular', 'qt_ing_enem', 'qt_ing_vg_remanesc',
       'qt_ing_25_29', 'qt_ing_30_34', 'qt_ing_35_39', 'qt_ing_40_49',
       'qt_ing_50_59'

In [70]:
###'qt_vg_total_diurno','qt_vg_total_noturno','qt_inscrito_total_diurno', 'qt_ing', 'qt_ing_25_29', 'qt_ing_30_34', 'qt_ing_40_49', 'qt_ing_50_59', 'qt_mat_60_mais', 'qt_mat_cornd', 'qt_conc_diurno', 'qt_conc_30_34', 'qt_conc_preta', 'qt_mat_amarela', 'qt_mat_indigena'

In [71]:
# Dropando colunas que não serão utilizadas
X = X.drop(['qt_vg_total_diurno',
 'qt_vg_total_noturno',
 'qt_inscrito_total_diurno',
 'qt_ing_25_29',
 'qt_ing_30_34',
 'qt_ing_40_49',
 'qt_ing_50_59',
 'qt_mat_60_mais',
 'qt_mat_cornd',
 'qt_conc_30_34',
 'qt_conc_preta',
 'qt_mat_amarela',
 'qt_mat_indigena',
 'co_regiao',
 'tp_categoria_administrativa',
 'co_ies',
 'qt_inscrito_total_noturno',
 'qt_ing_noturno',
 'qt_ing_vestibular',
 'qt_ing_enem',
 'qt_ing_vg_remanesc',
 'qt_ing_35_39',
 'qt_ing_parda',
 'qt_mat_50_59',
 'qt_conc_noturno',
 'qt_conc_35_39',
 'qt_conc_40_49',
 'qt_conc_cornd',
 'qt_aluno_deficiente',
 'qt_mat_rvpdef',
 'qt_ing_ativ_extracurricular', 'tp_organizacao_academica',
 'qt_conc_50_59',
 'qt_conc_amarela',
 'qt_mat_nacestrang'],axis=1)

In [72]:
X.columns

Index(['no_cine_area_geral_artes_e_humanidades',
       'no_cine_area_geral_ciencias_sociais,_comunicacao_e_informacao',
       'no_cine_area_geral_negocios,_administracao_e_direito',
       'no_cine_area_geral_saude_e_bem-estar', 'no_cine_area_geral_servicos',
       'tp_grau_academico_bacharelado', 'tp_grau_academico_tecnologico',
       'enade_faixa_1.5', 'enade_faixa_2.0', 'enade_faixa_3.0',
       'enade_faixa_4.0', 'enade_faixa_5.0', 'cpc_faixa_1.5', 'cpc_faixa_2.0',
       'cpc_faixa_3.0', 'cpc_faixa_4.0', 'cpc_faixa_5.0', 'qt_vg_total',
       'qt_vg_remanesc', 'qt_inscrito_total', 'qt_insc_vg_remanesc',
       'qt_ing_preta', 'qt_ing_amarela', 'qt_ing_cornd', 'qt_conc',
       'qt_ing_deficiente', 'qt_ing_reserva_vaga', 'qt_sit_trancada',
       'qt_sit_desvinculado', 'qt_apoio_social', 'qt_conc_apoio_social',
       'qt_ativ_extracurricular', 'co_curso', 'no_curso'],
      dtype='object')

In [73]:
X.shape

(1264, 34)

In [74]:
#Salvando os dados
X.to_csv('dados_processados/cursos-pos-processamento.csv',sep=';',index=False)